In [2]:
import pandas as pd
import numpy as np
import openai
import os

In [3]:
if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## Read data from file created in 'stavangerkommune/åarse_website.ipynb'

In [4]:

# Read the data
df = pd.read_feather('cleand.feather')
# remove rows  with missing embeddings
df = df[df.embedding.notnull()]
df.head(3).T

,4,5,6
id,8b853fb08f5e49e694d546728ea58ea8,44b23d5a563c49b8937fdadedde4a872,825e2f27b63147cd85f15e47391bb7c8
address,www.stavanger.kommune.no/om-stavanger-kommune/...,www.stavanger.kommune.no/naring-og-arbeidsliv/...,www.stavanger.kommune.no/politikk/finn-politik...
parent,5477ddfaf234486a9c42c8ab546608d0,79c8f7299b09416db82a92cba5e71783,18448b3cd3d74b10a54e270206e15d71
children,[],[],[]
header,Seriøsitetsbestemmelser Stavanger kommune,6.1 Brytningstid,Kort fortalt
paragraph_number,18,66,16
text,Seriøsitetsbestemmelser Stavanger kommune - ...,6.1 Brytningstid - Når denne planen skrives s...,Kort fortalt - Her finner du oversikt over al...
author,,,
date,03.06.2022,24.01.2023 12.29.21,
star_count,20,20,20


## Connect to Redis
Now that we have our Redis database running, we can connect to it using the Redis-py client. We will use the default host and port for the Redis database which is localhost:6379.

In [5]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

## Creating a Search Index in Redis
The below cells will show how to specify and create a search index in Redis. We will

1. Set some constants for defining our index like the distance metric and the index name
2. Define the index schema with RediSearch fields
3. Create the index

In [6]:
# Constants
VECTOR_DIM = len(df['embedding'][12]) # length of the vectors
VECTOR_NUMBER = len(df)                 # initial number of vectors
INDEX_NAME = "embeddings-index"           # name of the search index
PREFIX = "doc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [7]:
# Define RediSearch fields for each of the columns in the dataset
title = TextField(name="title")
url = TextField(name="url")
text = TextField(name="text")
date = TextField(name="date")

text_embedding = VectorField("content_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
fields = [title, url, text, date, text_embedding]

In [8]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
)

## Load Documents into the Index
Now that we have a search index, we can load documents into it. We will use the same documents we used in the previous examples. In Redis, either the Hash or JSON (if using RedisJSON in addition to RediSearch) data types can be used to store documents. We will use the HASH data type in this example. The below cells will show how to load documents into the index.

In [9]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for doc in records:
        key = f"{prefix}:{str(doc['id'])}"

        # create byte vectors for title and content
        content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # replace list of floats with byte vectors
        doc["content_vector"] = content_embedding

        client.hset(key, mapping = doc)

In [10]:
data = df.rename(columns={'header': 'title', 'address': 'url', 'text': 'text', 'date': 'date', 'embedding': 'content_vector'})[['id', 'url', 'title', 'text', 'date', 'content_vector']]
index_documents(redis_client, PREFIX, data)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Loaded 15879 documents in Redis search index with name: embeddings-index


## Simple Vector Search Queries with OpenAI Query Embeddings
Now that we have a search index and documents loaded into it, we can run search queries. Below we will provide a function that will run a search query and return the results. Using this function we run a few queries that will show how you can utilize Redis as a vector database.

In [11]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "embeddings-index",
    vector_field: str = "content_vector",
    return_fields: list = ["title", "url", "text", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
    print_results: bool = True,
) -> list[dict]:

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                            model="text-embedding-ada-002",
                                            )["data"][0]['embedding']

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    if print_results:
        for i, article in enumerate(results.docs):
            score = 1 - float(article.vector_score)
            print(f"{i}. {article.title} (Score: {round(score ,3) })")
    return results.docs

In [12]:
# For using OpenAI to generate query embedding
#results = search_redis(redis_client, 'Jeg er 20år, interessert i dans og musikk og ønsker å søke på et legat, Hvor skal jeg sende inn søknadden? Det er ikke rett, jeg skal sende inn søknad til legat.', k=10)
results = search_redis(redis_client, 'Jeg er en kvinne på 82år og lurer på Koronavaksine, nyheter', k=10)

0.  Ser nytten av vaksinen (Score: 0.859)
1.  - Vær forsiktig fram til vaksinen (Score: 0.857)
2.  Usikkert hvordan pandemien utvikler seg (Score: 0.857)
3.  - Vær forsiktig fram til vaksinen (Score: 0.856)
4.  Ber alle være forsiktige (Score: 0.855)
5.  Hvem som skal ta dose 4 (Score: 0.853)
6.  Finnøy (Score: 0.852)
7.  Sykehjemsbeboer døde få dager etter vaksine (Score: 0.851)
8.  De som er over 75 år kan få dose 4 (Score: 0.85)
9.  Sykehjemsbeboer døde få dager etter vaksine (Score: 0.85)


In [98]:
results[0]

Document {'id': 'doc:9cc1107931334f2580830fc5a174c3eb', 'payload': None, 'vector_score': '0.140718996525', 'title': ' Ser nytten av vaksinen', 'url': 'www.stavanger.kommune.no/nn/nyheter/koronadodsfall-ved-ramsvigtunet-sykehjem-og-bofellesskap/index.html', 'text': ' Ser nytten av vaksinen - Smittevernoverlege Ruth Midtgarden fremholder at selv om utbruddet er stort, så\ner dødeligheten lav i forhold til før vaksineringen.\n-Selv om mange av de som er smittet er vaksinert, så er sykeligheten og\ndødeligheten mye lavere nå enn i starten av pandemien. Så vi ser nytten av\nvaksineringen, sier Midtgarden.\nDe pårørende har samtykket til at Stavanger kommune informerer om dødsfallet.\nAv personvernhensyn vil det ikke bli gitt flere opplysninger om avdøde.\nSykehjemmet er stengt for besøk inntil videre. Dagsenteret er foreløpig stengt\nfram til 22. november. Alle friske beboere ved sykehjemmet har fått dose 3 av\nkoronavaksine samt influensavaksine.\n'}

In [63]:
results[7].text.replace('\n', '').replace('\xa0','')

' Hvem kan søke? - Alle lag og organisasjoner, idrettslag, musikkorps, velforeninger, borettslagm.m. kan søke om tilskudd til aktiviteter. Tilskuddet skal gå til aktiviteterfor hele befolkningen, men ha særlig søkelys på aktiviteter for barn og unge ikommunedelen.'

In [64]:
input_text = """As a helpfull Chatbot you should answer the User based on the Text provided in the chat. \nUser: Jeg er 20år, interessert i dans og musikk og ønsker å søke på et legat. \n Text: ' Hvem kan søke? - Formålet med legatet er å støtte seriøse kulturformål, i første rekke seriøs musikk og seriøs bildende kunst. Tilskuddsmidler fra legatets avkastning kan gis til større prosjekter og tiltak i Stavanger. Tiltak som ikke omfattes av andre kommunale tilskudds- og/eller finansieringsordninger vil normalt bli prioritert. Institusjoner, organisasjoner og enkeltpersoner i Stavanger kan søke. Det gis ikke midler til ordinær drift og programvirksomhet samt til utdanning.' \n Chatbot:""" 

In [78]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [83]:
response['choices'][0]['message']['content']

'The 2020 World Series was played at Globe Life Field, a baseball park in Arlington, Texas.'